In [0]:
from pyspark.sql.functions import col, to_timestamp, when, concat_ws

In [0]:
#ler bronze
df_partidas_bronze = spark.table("project_data_football_bronze.partidas_rodada")

In [0]:
# transformar para silver
df_partidas_silver = df_partidas_bronze.select(
    
    # Identificação
    col("partida_id").cast("long"),
    col("campeonato_id").cast("long"),
    col("rodada").cast("int"),

    # Clubes
    col("clube_casa_id").cast("long"),
    col("clube_visitante_id").cast("long"),
    col("clube_casa_posicao").cast("int"),
    col("clube_visitante_posicao").cast("int"),
    
    # Placar
    col("placar_oficial_mandante").alias("gols_casa").cast("int"),
    col("placar_oficial_visitante").alias("gols_visitante").cast("int"),
    
    # Aproveitamento
    concat_ws(",", col("aproveitamento_mandante")).alias("aproveitamento_mandante"),
    concat_ws(",", col("aproveitamento_visitante")).alias("aproveitamento_visitante"),
    
    # Informações da partida
    col("local").alias("estadio"),
    col("transmissao"),
    col("status_transmissao_tr").alias("status_transmissao"),
    col("status_cronometro_tr").alias("status_cronometro"),
    col("periodo_tr").alias("periodo"),
    
    # Datas
    to_timestamp("partida_data").alias("data_partida"),
    to_timestamp("timestamp").alias("data_atualizacao"),
    
    # Flags
    col("valida").cast("boolean"),
    
    # Metadata
    col("dt_ingestao")
)


In [0]:
# colunas análiticas extras
df_partidas_silver = df_partidas_silver.withColumn(
    "resultado",
    when(col("gols_casa") > col("gols_visitante"), "MANDANTE")
    .when(col("gols_casa") < col("gols_visitante"), "VISITANTE")
    .otherwise("EMPATE")
)

In [0]:
#salvar
df_partidas_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("project_data_football_silver.fato_partida")
